In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your data from the 'Cleaned_data.txt' file into a DataFrame
data = pd.read_csv('./Cleaned_Data.txt', sep=', ',  header=None, names=['Image1', 'Image2', 'Label'])

# Split the data into training (70%), validation (15%), and testing (15%) sets
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Now you have your data split into training, validation, and testing sets
# You can access the data and labels as follows:
train_images1, train_images2, train_labels = train_data['Image1'], train_data['Image2'], train_data['Label']
val_images1, val_images2, val_labels = val_data['Image1'], val_data['Image2'], val_data['Label']
test_images1, test_images2, test_labels = test_data['Image1'], test_data['Image2'], test_data['Label']


/var/folders/0y/926bml0x27j90kg_6jr1s2h00000gn/T/ipykernel_25345/2401875523.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('./Cleaned_Data.txt', sep=', ',  header=None, names=['Image1', 'Image2', 'Label'])


In [1]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [20]:
import numpy as np
import cv2
import pandas as pd

# Load a pre-trained Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect and align faces in an image
def detect_and_align_faces(image_path):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image to grayscale for face detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    aligned_faces = []

    # Loop through detected faces
    for (x, y, w, h) in faces:
        # Extract the detected face region
        face = image[y:y+h, x:x+w]

        # Resize the face to a common size for alignment (e.g., 160x160 pixels)
        aligned_face = cv2.resize(face, (160, 160))

        aligned_faces.append(aligned_face)

    return aligned_faces

# Read the original data from the text file
data = pd.read_csv('Cleaned_Data.txt', sep=', ',  header=None, names=['Image1', 'Image2', 'Label'])

# Create a new DataFrame for the processed data
new_data = pd.DataFrame(columns=['Image1', 'Image2', 'Label'])

# Process each row in the original data
for index, row in data.iterrows():
    image1_path = row['Image1']
    image2_path = row['Image2']
    label = row['Label']

    # Detect and align faces for Image1 and Image2
    aligned_faces1 = detect_and_align_faces(image1_path)
    aligned_faces2 = detect_and_align_faces(image2_path)

    # Create new rows in the DataFrame for each aligned face pair
    for face1, face2 in zip(aligned_faces1, aligned_faces2):
        new_data = new_data.append({'Image1': 'path_to_aligned_face1.jpg',
                                    'Image2': 'path_to_aligned_face2.jpg',
                                    'Label': label}, ignore_index=True)

# Save the processed data to a new text file
new_data.to_csv('Processed_data.txt', sep='\t', header=False, index=False)


TypeError: 'numpy._DTypeMeta' object is not subscriptable